In [6]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/?ids=4JpKVNYnVcJ8tuMKjAj50A,2NRANZE9UCmPAS5XVbXL40,24JygzOLM0EmRQeGtFcIcG

#https://www.kaggle.com/aeryan/spotify-music-analysis

#https://www.kaggle.com/cihanoklap/top-songs-on-spotify-what-makes-them-popular

In [1]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "Charts CSV/regional-global-daily-latest.csv"
spotify_data_global = pd.read_csv(path)
spotify_data_global.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
2,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
3,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
4,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE


In [4]:
spotify_data_global.to_csv("clean_global.csv", header=False, index=False)


In [12]:
path_clean = "clean_global.csv"
spotify_data_global_clean = pd.read_csv(path_clean)
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz


In [5]:
# token = "BQAEIQPWvNjgZD41CmvBSrkEvOTxVYxWU1l9GaAFb79KuLLanpGlLTccAsdvVz5yK-xzOFVzHnnDYARd-eZbZGKsuQU0Yt8MofmTOiPCW3CRv6PZYmcxHro-BQ25W6gUcbyd6YTxWEccl9-1UP8SNAArj7IjeQ"
# token = "BQC7SbbuUwwebllY8WlsJYjDKwpMeTF8twCsti47Or50rxWXZ0JOYs7IXIrDOb6LnhE5YPUu_nUkkjTma-zh6xPjf_j7AtCIEk4qOdQu8i_wvvjie5Bp1xANNhhhjusybp6spB2ZFhbJ-0dTfVE"
# token = "BQB1hjVmwHas7EQTBD5SgV0fYBJFhkQdLjo6QV9qMxtwM-KWxVOcVt1f1934NQ2wPT4xZ27TQVpUsQV5JG-GR9-GJoE1iAvqR32bHL4ttyRNRgIChRd4sCfUoFeSMhodlYUUv5Tys9sVJk-fdWNXNg"
#token="BQAqFieWd9Vs7_PadOTnr5aa5Z718R_mmO6WACUnALMgjtzORXuE1eV6wxBonSkIlbKt924WCoIdF2Ia9QSgFlKzw2VONVEbgzTYTMUobMQTP4CSybOuLgCyAc8dasFXh8ycBwfmtLKvrHb5D9v7c3t_DxahFixpPZd-TsBPoCbRgw"
token = "BQCS-50SGGuFtYv4HE0mMt52SWYG4KWPkPROJiVSc1JsOtIbEnhknMeuDnbq7BvSoOIdsmsfy7SGfLEB3Bpvn1p4-eouNRD2zTNZyt3CqYNgZa-elzG1OvX5XsMfyNrTphObI7icW220YJYE60AzBVLygw6hQo3-ZXgkD5DFEmMzFym1E-o9Nr09wT4-9SWRs8Gh_tTKCskmfRjE625eHyLVE8Oz0GAasIXGz40F0f6OsqTVwfE46fiUUir1BMbcsYcre7BdvjJoDGwhSSx3FK1wI3GoDXA"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopGlobal
# collection.drop()

rows=len(spotify_data_global_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    #Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist = spotify_data_global_clean.iloc[row,0]

    
    response = requests.get(f'https://api.spotify.com/v1/search?q={artist}&type=artist', headers=headers_dict).json()
#     print(response)
    artists = response["artists"]
    

    danceability = items["audio_features"]["danceability"][0]
    energy = items["audio_features"]["energy"][0]
    key = items["audio_features"]["liveness"][0]
    loudness = items["audio_features"]["key"][0]
    acousticness = items["audio_features"]["acousticness"][0]
    instrumentalness = items["audio_features"]["instrumentalness"][0]
    liveness = items["audio_features"]["liveness"][0]
    valence = items["audio_features"]["valence"][0]
    tempo = items["audio_features"]["tempo"][0]
    track_href = items["audio_features"]["track_href"][0]
        
        
    global_dict={'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': followers,
                 'Acousticness': image,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)
   

NameError: name 'spotify_data_global_clean' is not defined